In [1]:
import easygui
import os
import cv2
faceCascade = cv2.CascadeClassifier("face.xml")
try:
    name = easygui.enterbox()
except:
    name = "random"
try:
    os.mkdir("datasetf")
except:
    pass
try:
    os.mkdir("datasetf"+f"\\"+name)
except:
    pass
video_capture = cv2.VideoCapture(0)
new_path = "datasetf\\"+name

In [2]:
name

'simran'

In [3]:
i = 0
while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(100, 100), flags=cv2.CASCADE_SCALE_IMAGE)
    for (x, y, w, h) in faces:
        i=i+1
        cv2.imwrite(new_path+"\\"+name +'_'+ str(i) + ".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    try:
        cv2.imshow('Video', frame)
    except:
        continue
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if i+1>200:
        break
video_capture.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import os
from PIL import Image 
import pickle
import numpy as np
face_dict = {}
def get_images_and_labels(path):
    global face_dict
    names=os.listdir("datasetf")
    image_paths=[]
    new_path="datasetf"
    for name in names:
        image_paths.append([os.path.join(new_path+"\\"+name, f) for f in os.listdir(new_path+"\\"+name)])
    
 # images will contains face images
    images = []
 # labels will contains the label that is assigned to the image
    labels = []
    nbr = 0
    i = 0
    j = 0
    k = 0
    for name in names:
        for image_path in image_paths[j]:
             # Read the image and convert to grayscale
            image_pil = Image.open(image_path).convert('L')
             # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            face_dict[j] = name
            #nbr=int(''.join(str(ord(c)) for c in nbr))
            
             # Detect the face in the image
            faceCascade = cv2.CascadeClassifier('face.xml')
            faces = faceCascade.detectMultiScale(image)
             # If face is detected, append the face to images and the label to labels
            for (x, y, w, h) in faces:
                images.append(image[y: y + h, x: x + w])
                labels.append(j)
                #cv2.imshow("Adding faces to traning set...", image[y: y + h, x: x + w])
                cv2.waitKey(10)
            i = i+1
        j = j+1
    return images,labels

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#file_rename(path,names)
images, labels = get_images_and_labels('datasetf\\random')
recognizer = cv2.face.LBPHFaceRecognizer_create()
save_obj(face_dict,"trainer\\trained_faces")
recognizer.train(images, np.array(labels))
recognizer.save('trainer\\trainer.yml')
cv2.destroyAllWindows()

# Face Prediction

In [7]:
import cv2
import pickle
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer\\trainer.yml')
cascadePath = "face.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
nbr_predicted, conf=0,0
with open("trainer\\trained_faces.pkl",'rb') as f:
    face_dict = pickle.load(f)
# face_dict=load_obj("trainer\\trained_faces")
cam = cv2.VideoCapture(0)
pre_detect = 0
while True:
    ret, im = cam.read()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(100, 100), flags=cv2.CASCADE_SCALE_IMAGE)
    for(x,y,w,h) in faces:
        nbr_predicted, conf = recognizer.predict(gray[y:y+h,x:x+w])
        #print(round(100 - conf, 2))
        cv2.rectangle(im,(x-50,y-50),(x+w+50,y+h+50),(225,0,0),2)
        label = face_dict[nbr_predicted]
        if(conf<70):
            cv2.putText(im,str(label),(x,y+h),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
#           detect = 1   
        else :
            cv2.putText(im,str("UNKNOWN"),(x,y+h),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
            detect = 0
        cv2.putText(im,"press esc to exit",(x-45,y-40),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,0),2)
#         if pre_detect != detect:
#             aio.send('foo',detect)
#             pre_detect = detect
    cv2.imshow('im',im)
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break
cam.release()
cv2.destroyAllWindows()